In [ ]:
from autumn.projects.sm_covid2.common_school.calibration import get_bcm_object, plot_model_fit
from autumn.projects.sm_covid2.common_school.runner_tools import optimise_model_fit
from autumn.projects.sm_covid2.common_school.calibration_plots.opti_plots import plot_opti_params

from scipy.stats import qmc

In [ ]:
iso3 = "FRA"
n_samples = 2

In [ ]:
bcm = get_bcm_object(iso3, "main")

In [ ]:
lhs_sampled_params = [p for p in bcm.priors if p != "random_process.delta_values"]  

In [ ]:
d = len(lhs_sampled_params)
sampler = qmc.LatinHypercube(d=d)
regular_sample = sampler.random(n=n_samples)

In [ ]:
l_bounds = [bcm.priors[p].bounds()[0] for p in lhs_sampled_params]
u_bounds = [bcm.priors[p].bounds()[1] for p in lhs_sampled_params]

In [ ]:
sample = qmc.scale(regular_sample, l_bounds, u_bounds)
sample_as_dicts = [{p: sample[i][j] for j, p in enumerate(lhs_sampled_params)} for i in range(n_samples)]

In [ ]:
import yaml

with open('test_dump.yml', 'w') as outfile:
    yaml.dump(sample_as_dicts, outfile)

In [ ]:
best_params = {}
for j, sample_dict in enumerate(sample_as_dicts):
    best_p, _ = optimise_model_fit(bcm, 200, 500, suggested_start=sample_dict)
    best_params[j] = best_p

In [ ]:
with open('test_dump_best.yml', 'w') as outfile:
    yaml.dump(best_params, outfile)

In [ ]:
f = plot_model_fit(bcm, best_params[0], "name")

In [ ]:
fig = plot_opti_params(sample_as_dicts, best_params, bcm)